In [98]:
import numpy as np
import random

class Person:
    def __init__(self):
        self.recover_prob = 0.2
        self.die_prob = 0.05
        self.init_sick_prob = 0.1
        self.recovered = False
        self.dead = False
        self.sick = False
        self.infect_others_prob = 0.05 #infectin rate
        self.average_meetups = 10
        self.days_sick=0
        
        self.init_sick_or_not()
        
    def init_sick_or_not(self):
        """When created each person starts as either sick or healthy"""
        prob = random.random() #The returned value is 0-1: https://www.w3schools.com/python/ref_random_random.asp
        if prob <= self.init_sick_prob:
            # This should yield that about 10% of the people created are sick, the rest healthy
            self.sick = True
        else:
            self.sick = False
    
    def day_passes(self, population, init_scenario=False):
        """This method describes what happends to each person each day"""
        
        # If a person is sick, they can infect others.
        #prob = random.random()
        if self.sick:
            self.days_sick+=1
            if init_scenario==False: #Why is it "False"?
                self.infect_others(population)
        
        # Will person recover?
        prob = random.random()
        if prob <= self.recover_prob and self.sick == True:
            # Person has rehabilitated and is now healthy!
            self.sick = False
            self.recovered = True 
        
        # If person is still sick they might die
        prob = random.random()
        if prob <= self.die_prob and self.sick == True:
            self.dead = True
            self.sick = False
            
    def infect_others(self, population):
        person_encounters = random.sample(range(population.size), k= self.average_meetups)
        for person_id in person_encounters:
            prob = random.random()
            if prob <= self.infect_others_prob and population[person_id].recovered==False and population[person_id].dead==False:
            # Person has rehabilitated and is now healthy!
                population[person_id].sick = True
             

In [101]:
class Village:
    def __init__(self, init_population_size):
        self.population = np.empty(init_population_size, Person)#https://www.w3resource.com/numpy/array-creation/empty.php. Question: Class Person as dtype? Why isn't it according to the syntax empty(size, dtype=Person)
        self.generate_inhabitants()
    
    def generate_inhabitants(self):
        """Generates the population"""
        for i in range(self.population.size):
            self.population[i] = Person()
        
    def advance_days(self, init_scenario = False):
        """Counts the status of the citizens in the community"""
        people_sick = 0
        people_recovered = 0
        people_dead = 0
        for person in self.population:
            if person.sick == True:
                people_sick += 1
            elif person.recovered == True:
                people_recovered += 1
            elif person.dead == True:
                people_dead += 1
            
            person.day_passes(self.population, init_scenario)

        return people_sick, people_recovered, people_dead
    
    def start_simulation(self):
        """This function controls the simulation and what happends in a day"""
        current_day = 0
        
        people_sick, people_recovered, people_dead = self.advance_days(init_scenario = True)
        while people_sick:
            print(f"By day {current_day} {people_sick} people are sick, {people_dead} are dead and {people_recovered} has recovered")
            current_day += 1
            people_sick, people_recovered, people_dead = self.advance_days()
            Patient_with_duration=[population.days_sick in self.population if self.sick]
        
        people_unaffected = self.population.size - (people_sick + people_recovered + people_dead)
        print(f"\nBy day {current_day} {people_sick} people are sick, {people_dead} are dead and {people_recovered} has recovered. {people_unaffected} people were never in contact with the virus")
        print("The village has recovered and the virus is eliminated!")
        print(self.population[999])

SyntaxError: expected 'else' after 'if' expression (613927197.py, line 37)

In [100]:
def main():
    pop_size = 1000
    village = Village(pop_size)
    village.start_simulation()

main()

AttributeError: 'Village' object has no attribute 'days_sick'